# Validación Numérica de Curvas de Rotación
## Teoría del Espacio Dinámico Rotacional (EDR)

Este notebook realiza la validación numérica comparando:

1. Curva de rotación observada (real o sintética)
2. Curva de rotación Newton/GR
3. Predicción modificada por EDR

El modelo EDR usa:
- Corrección radial:
  \[ a_{EDR}(r) = k_{flow} \, \eta \, \Omega_{flow}(r)^2 \, f(r) \]
- Parámetros ajustables: \(k_{flow}, \eta, \Omega_0\)
- Ajuste por mínimos cuadrados


## 1. Imports


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import simps

## 2. Funciones físicas básicas
Modelo Newton/GR para masa baryónica en disco exponencial.


In [ ]:
# Masa superficial
def Sigma(r, Sigma0=500, Rd=3.0):
    return Sigma0 * np.exp(-r / Rd)

# Masa encerrada
def M_enc(r):
    rs = np.linspace(0.01, r, 500)
    integrand = 2 * np.pi * rs * Sigma(rs)
    return simps(integrand, rs)

# Velocidad Newton/GR
def v_GR(r):
    G = 4.30091e-6
    return np.sqrt(G * M_enc(r) / r)

## 3. Modelo EDR
Usamos un perfil de flujo:
\[
\Omega_{flow}(r) = \Omega_0 \, e^{-r/R_\Omega}
\]
Y corrección a la aceleración:
\[
a_{EDR} = k_{flow} \, \eta \, \Omega_{flow}(r)^2
\]


In [ ]:
def Omega_flow(r, Omega0, R_Omega):
    return Omega0 * np.exp(-r / R_Omega)

def a_EDR(r, kflow, eta, Omega0, R_Omega):
    return kflow * eta * Omega_flow(r, Omega0, R_Omega)**2

def v_EDR(r, kflow, eta, Omega0, R_Omega):
    return np.sqrt(v_GR(r)**2 + r * a_EDR(r, kflow, eta, Omega0, R_Omega))

## 4. Cargar o generar datos observacionales
Si no existe archivo, generamos datos sintéticos.


In [ ]:
def generate_mock():
    r = np.linspace(0.5, 20, 40)
    v0 = v_GR(r)
    # Añadimos una componente plana típica de galaxias reales
    v_obs = np.sqrt(v0**2 + 80**2)
    noise = np.random.normal(0, 4, len(r))
    return r, v_obs + noise

r_data, v_obs = generate_mock()

## 5. Función de error para ajuste EDR


In [ ]:
def chi2(params):
    kflow, eta, Omega0, R_Omega = params
    v_model = v_EDR(r_data, kflow, eta, Omega0, R_Omega)
    return np.sum((v_model - v_obs)**2)

initial_guess = [0.01, 0.02, 0.1, 5.0]

### 6. Ejecutar ajuste numérico


In [ ]:
sol = minimize(chi2, initial_guess, method='Nelder-Mead')
sol.x

## 7. Graficar comparación final


In [ ]:
kflow, eta_opt, Omega0_opt, R_O_opt = sol.x

r_plot = np.linspace(0.5, 20, 300)
v_gr_plot = v_GR(r_plot)
v_edr_plot = v_EDR(r_plot, kflow, eta_opt, Omega0_opt, R_O_opt)

plt.figure(figsize=(8,6))
plt.scatter(r_data, v_obs, label='Observado', color='black')
plt.plot(r_plot, v_gr_plot, label='Newton/GR', linestyle='--')
plt.plot(r_plot, v_edr_plot, label='EDR ajustado', linewidth=2)
plt.xlabel('r (kpc)')
plt.ylabel('v (km/s)')
plt.legend()
plt.title('Validación EDR — Curvas de Rotación')
plt.grid(True)
plt.show()

## 8. Parámetros finales


In [ ]:
print('k_flow =', kflow)
print('eta =', eta_opt)
print('Omega0 =', Omega0_opt)
print('R_Omega =', R_O_opt)